In [1]:
import pandas as pd
import numpy as np
import ast  # Add this import for literal_eval
import requests
from PIL import Image as PILImage
from io import BytesIO
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import normalize

2024-03-18 16:11:39.108068: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 16:11:39.108247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 16:11:39.285036: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
csv_file_path = '/kaggle/input/ir-ass2/A2_Data.csv'
df = pd.read_csv(csv_file_path)

h = df[['Unnamed: 0', 'Image']]

In [6]:
# Assuming df is your DataFrame containing 'Unnamed: 0' and 'Review Text' columns

# Extracting 'Unnamed: 0' and 'Review Text' columns as a list of tuples
review_tuples = df[['Unnamed: 0', 'Review Text']].to_numpy()

# Creating a dictionary comprehension to directly generate review_dict
review_dict = {row[0]: row[1] for row in review_tuples}

# Extracting all review texts into a list using list comprehension
all_text = [row[1] for row in review_tuples]


In [11]:
array = h.to_numpy()

In [13]:
# Assuming 'array' is a list of tuples where each tuple contains two elements: (key, string_of_links)

# Define an empty dictionary to store the links
all_links = {}

# Iterate over each tuple in 'array'
for key, links_string in array:
    # Extract links from the string representation, removing leading and trailing characters, and split by comma
    image_list = links_string.strip('[]').split(', ')
    # Assign the extracted links to the corresponding key in the dictionary
    all_links[key] = image_list


# print(len(all_links))

In [14]:
# Assuming 'all_links' is a dictionary where keys are image names and values are corresponding review keys,
# and 'review_dict' is a dictionary where keys are review keys and values are review texts.

# Define an empty dictionary to store the mapping of image names to review texts
image_to_text_dict = {}

# Iterate over each key-value pair in 'all_links'
for image_name, review_key in all_links.items():
    # Iterate over each review key associated with the current image name
    for key in review_key:
        # Retrieve the review text associated with the current review key
        review_text = review_dict.get(key)
        # If review text exists, add a mapping from the image name to the review text
        if review_text:
            image_to_text_dict[image_name.strip("'").strip()] = review_text

# The final dictionary 'image_to_text_dict' contains image names mapped to corresponding review texts.


# print(image_to_text_dict['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg'])

In [15]:
# Assuming 'review_dict' is a dictionary where keys are review keys and values are review texts,
# and 'all_links' is a dictionary where keys are review keys and values are lists of image names.

# Define an empty dictionary to store the mapping of review texts to image names
text_to_image_dict = {}

# Iterate over each key-value pair in 'review_dict'
for review_key, review_text in review_dict.items():
    # Retrieve the list of image names associated with the current review key
    image_names = all_links.get(review_key, [])
    # Add a mapping from the review text to the list of image names
    text_to_image_dict[review_text] = image_names

# The final dictionary 'text_to_image_dict' contains review texts mapped to corresponding lists of image names.


# print(text_to_image_dict['Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.'])

In [9]:
# Assuming 'all_links' is a dictionary where keys are review keys and values are lists of image names.

# Create an empty list to store all URLs
all_urls = []

# Iterate over each value (list of image names) in 'all_links'
for image_names in all_links.values():
    # Extend the 'all_urls' list with the contents of the current list of image names
    all_urls.extend(image_names)

# The 'all_urls' list now contains all the URLs from the 'all_links' dictionary.

        
# print(image_to_text_dict[all_url[0].strip("'").strip()])
# print(text_to_image_dict[all_text[0]])

In [10]:
import requests
from PIL import Image as PILImage, ImageEnhance
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from io import BytesIO
import random
import numpy as np

def fetch_image_from_url(url):
    # Ensure the URL starts with 'http'
    if not url.startswith('http'):
        url = 'http://' + url.strip("'")
    
    # Fetch the image from the URL
    response = requests.get(url)
    img = PILImage.open(BytesIO(response.content))
    return img

def resize_image(img):
    # Resize the image to the desired size
    img = img.resize((224, 224))
    return img

def apply_random_transformations(img):
    # Apply random rotation
    random_rotation_angle = random.uniform(-45, 45)
    img = img.rotate(random_rotation_angle)

    # Apply random flips (horizontal and vertical)
    if random.choice([True, False]):
        img = img.transpose(PILImage.FLIP_LEFT_RIGHT)
    if random.choice([True, False]):
        img = img.transpose(PILImage.FLIP_TOP_BOTTOM)

    # Adjust brightness
    brightness_factor = random.uniform(0.8, 1.2)
    img = ImageEnhance.Brightness(img).enhance(brightness_factor)

    # Adjust exposure
    exposure_factor = random.uniform(0.8, 1.2)
    img = ImageEnhance.Contrast(img).enhance(exposure_factor)
    return img

def preprocess_image(img):
    # Convert the image to an array, preprocess it, and return
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def preprocess_image_from_url(url):
    img = fetch_image_from_url(url)
    img = resize_image(img)
    img = apply_random_transformations(img)
    img_array = preprocess_image(img)
    return img_array


In [11]:
# Step 1b: Use Pre-trained VGG16
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import numpy as np

def extract_features(img_array):
    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False)

    # Preprocess the image array
    img_array_preprocessed = preprocess_input(img_array)

    # Extract features using the VGG16 model
    features = base_model.predict(img_array_preprocessed)

    return features


In [12]:
# Step 1c: Normalize Extracted Features
from sklearn.preprocessing import normalize

def normalize_features(features):
    # Normalize the features along the last axis using L2 normalization
    normalized_features = normalize(features, axis=-1, order=2)
    return normalized_features


In [13]:
image_features_list = []

In [14]:
# processing all images of CSV file and saving normalized_features of each image into image_features_list
from PIL import UnidentifiedImageError

image_features_list = []  # Initialize a list to store the normalized features

for i, img_url in enumerate(all_url):
    try:
        img_url = img_url.strip("'")
        print(f"Normalized Features for Image {i+1}:")
        
        # Preprocess the image
        img_array = preprocess_image(img_url)
        
        # Extract features from the preprocessed image
        extracted_features = extract_features(img_array)
        
        # Normalize the extracted features
        normalized_features = normalize_features(extracted_features)

        # Append the normalized features to the list
        image_features_list.append(normalized_features)
    except (ValueError, IndexError, requests.exceptions.RequestException, UnidentifiedImageError) as e:
        print(f"Error processing image from {img_url}: {e}")


Normalized Features for Image 1:







1/1 [==============================] - 2s 2s/step

Normalized Features for Image 2:

1/1 [==============================] - 1s 794ms/step

Normalized Features for Image 3:

1/1 [==============================] - 1s 1s/step

Normalized Features for Image 4:

1/1 [==============================] - 1s 618ms/step

Normalized Features for Image 5:


1/1 [==============================] - 1s 600ms/step

Normalized Features for Image 6:


1/1 [==============================] - 1s 666ms/step

Normalized Features for Image 7:

1/1 [==============================] - 1s 938ms/step

Normalized Features for Image 8:

1/1 [==============================] - 1s 786ms/step

Normalized Features for Image 9:

1/1 [==============================] - 1s 708ms/step

Normalized Features for Image 10:

1/1 [==============================] - 1s 670ms/step

Normalized Features for Image 11:

1/1 [==============================] - 1s 598ms/step

Normalized Features for Imag

In [15]:
# Save image features using pickle
import pickle

# Define the file path to save the image features
file_path = 'image_features.pkl'

# Open the file in binary write mode and use a more descriptive variable name
with open(file_path, 'wb') as pickle_file:
    # Serialize and save the image features list to the file
    pickle.dump(image_features_list, pickle_file)

# Confirm the successful saving of the file
print(f"Image features have been successfully saved to '{file_path}'.")


In [16]:
import pickle

# Define the file path from which to load the image features
file_path = 'image_features.pkl'

# Open the file in binary read mode and use a more descriptive variable name
with open(file_path, 'rb') as pickle_file:
    # Load the serialized image features from the file
    loaded_features = pickle.load(pickle_file)

# Confirm the successful loading of the file
print(f"Image features have been successfully loaded from '{file_path}'.")

# Now 'loaded_features' contains the deserialized image features

#print(loaded_features)

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import pickle

In [18]:
# Step 2a: Text Pre-processing
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Rishu

[nltk_data]     Rathor\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!

[nltk_data] Downloading package punkt to C:\Users\Rishu

[nltk_data]     Rathor\AppData\Roaming\nltk_data...

[nltk_data]   Package punkt is already up-to-date!

[nltk_data] Downloading package wordnet to C:\Users\Rishu

[nltk_data]     Rathor\AppData\Roaming\nltk_data...

[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [20]:
def preprocess_text(text):
    # Check if the value is a string
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        
        # Tokenization
        tokens = nltk.word_tokenize(text)
        
        # Remove punctuation and stop words, and apply stemming and lemmatization
        filtered_tokens = [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens if token.isalnum() and token not in stop_words]
        
        # Join the tokens back into a single string
        preprocessed_text = ' '.join(filtered_tokens)
        
        return preprocessed_text
    else:
        return ''

In [22]:
# Apply text pre-processing to the 'text' column and create a new 'preprocessed_text' column
df['preprocessed_text'] = df['Review Text'].apply(preprocess_text)
# print(df)

In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

def calculate_tfidf_scores(corpus, max_features=5000):
    # Step 1: Vectorize the corpus to obtain document-term matrix
    vectorizer = CountVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(corpus)

    # Step 2: Transform the count matrix to TF-IDF representation
    tfidf_transformer = TfidfTransformer()
    tfidf_matrix = tfidf_transformer.fit_transform(X)

    # Extract feature names from the vectorizer
    selected_features = vectorizer.get_feature_names_out()

    return tfidf_matrix, selected_features

preprocessed_texts = df['preprocessed_text'].tolist()

# Calculate TF-IDF scores
tfidf_matrix, selected_features = calculate_tfidf_scores(preprocessed_texts)

# Now, tfidf_matrix contains the TF-IDF scores, and selected_features contains the selected feature words.


In [27]:
from scipy.sparse import csr_matrix
# Convert the dense matrix to a sparse matrix in CSR format
tfidf_score = csr_matrix(tfidf_matrix)
# print(tfidf_score)
# print(df)

In [28]:
# Save preprocessed text and TF-IDF scores using pickle
with open('preprocessed_text.pkl', 'wb') as file:
    pickle.dump(df['preprocessed_text'], file)

with open('tfidf_score.pkl', 'wb') as file:
    pickle.dump(tfidf_score, file)

In [31]:
with open('preprocessed_text.pkl','rb') as file:
    loaded_preprocessed_text = pickle.load(file)
# print(loaded_preprocessed_text)

with open('tfidf_score.pkl','rb') as file:
    loaded_tfidf_score = pickle.load(file)
# print(loaded_tfidf_score)

In [79]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between input image features and all image features
image_features_array = np.array(image_features_list)  # Convert image features list to array
num_samples_image, _, height, width, channels = image_features_array.shape  # Get dimensions of the image features array
image_features_2d = image_features_array.reshape(num_samples_image, -1)  # Reshape image features array to 2D

num_samples_input, _, _, _, _ = input_image_features.shape  # Get dimensions of the input image features array
input_image_features_2d = input_image_features.reshape(num_samples_input, -1)  # Reshape input image features array to 2D

image_similarity_scores = cosine_similarity(input_image_features_2d, image_features_2d)  # Compute cosine similarity scores
top_similar_images_indices = image_similarity_scores.argsort()[0][-3:][::-1]  # Get indices of top similar images

print("USING IMAGE RETRIEVAL")
for i in top_similar_images_indices:
    print("Image URL:", all_url[i])  # Print top similar image URL
    print("Review:", image_to_text_dict[all_url[i].strip("'").strip()])  # Print review corresponding to the image
    print("Similarity score:", image_similarity_scores[0][i])  # Print similarity score
    print("===========================================================================================================")

# Calculate cosine similarity between input review TF-IDF scores and all reviews' TF-IDF scores
tfidf = np.array(tfidf_matrix)  # Convert TF-IDF matrix to array
preprocessed_texts = [input_review]  # Create a list containing the input review
input_tfidf_matrix, _ = calculate_tfidf_scores(preprocessed_texts)  # Calculate TF-IDF scores for the input review

text_similarity_scores = cosine_similarity(input_tfidf_matrix, tfidf)  # Compute cosine similarity scores
top_similar_reviews_indices = text_similarity_scores.argsort()[0][-3:][::-1]  # Get indices of top similar reviews

print("USING TEXT RETRIEVAL")
for i in top_similar_reviews_indices:
    print("Review Text:", all_text[i])  # Print top similar review text
    print("Image URL:", text_to_image_dict[all_text[i]])  # Print image URL corresponding to the review
    print("Similarity score:", text_similarity_scores[0][i])  # Print similarity score
    print("===========================================================================================================")

# Save results using Pickle
result_data = {
    'top_similar_images_indices': top_similar_images_indices,  # Store indices of top similar images
    'image_similarity_scores': image_similarity_scores,  # Store image similarity scores
    'top_similar_reviews_indices': top_similar_reviews_indices,  # Store indices of top similar reviews
    'text_similarity_scores': text_similarity_scores  # Store text similarity scores
}

with open('results.pkl', 'wb') as result_file:  # Open file to save results
    pickle.dump(result_data, result_file)  # Save result data to file


1/1 [==============================] - 2s 2s/step

USING IMAGE RETRIEVAL

Image URL: 'https://images-na.ssl-images-amazon.com/images/I/51hY-Gay44L._SY88.jpg'

Review: I have worked in radio for over 12 years as well as voiceover.  My main microphone has been the big brother to this one.  Most radio studios you enter use the RE20.  That being said when I recieved this RE 320 I was blown away.  I have done my radio show with it exclusively for the past 8 months and love it!  This mic ROCKS!  This microphone has XLR connections so be sure to use good mixer with those standard connections for a top quality mic like this.  When you first get this microphone you immediately notice the professional look and feel to it.  It is all that you would imagine it to be. Simply put this mic is top quality and produces great sound even without processing.  5 STARS!

similarity score: 0.2464929


Image URL: 'https://images-na.ssl-images-amazon.com/images/I/616BUvHEUaL._SY88.jpg'

Review: Purchased after

In [35]:
import pandas as pd

# Load the pre-processed data
file_path = "/kaggle/input/ir-ass2/A2_Data.csv"  # Provide the correct file path
df = pd.read_csv(file_path)

# Drop rows with NaN values in the 'Review Text' column
df.dropna(subset=['Review Text'], inplace=True)

# Display the first few rows of the DataFrame to verify changes
print(df.head())



ValueError: operands could not be broadcast together with shapes (1,1639) (1,1000) 

In [ ]:
# User input
image_url = input("Enter the image URL: ")
review_text = input("Enter the review text: ")
​


In [ ]:
# Define a function to retrieve input image features
def get_input_image_features(image_url):
    """
    Function to retrieve features for the input image.
    Args:
    - image_url (str): URL or path of the input image.
    Returns:
    - input_image_features: Features extracted from the input image.
    """
    # Add your implementation to extract features from the input image
    # This is a placeholder comment for implementation

# Define a function to calculate image similarity
def calculate_image_similarity(input_image_features, df):
    """
    Function to calculate image similarity.
    Args:
    - input_image_features: Features extracted from the input image.
    - df (DataFrame): DataFrame containing image features.
    Returns:
    - similarity_scores: Similarity scores between input image and images in the DataFrame.
    """
    # Add your implementation to calculate image similarity
    # Placeholder comment for implementation

# Define a function to calculate review similarity
def calculate_review_similarity(review_text, df):
    """
    Function to calculate review similarity.
    Args:
    - review_text (str): Text of the input review.
    - df (DataFrame): DataFrame containing review texts.
    Returns:
    - similarity_scores: Similarity scores between input review and reviews in the DataFrame.
    """
    # Add your implementation to calculate review similarity
    # Placeholder comment for implementation

# Load pre-processed data
df = pd.read_csv("/kaggle/input/ir-ass2/A2_Data.csv")

# Drop rows with NaN values in 'Review Text' column
df = df.dropna(subset=['Review Text'])

# Compute feature vector for input image
input_image_features = get_input_image_features(image_url)

if input_image_features is not None:
    # Calculate image similarity
    df['Similarity_Images'] = calculate_image_similarity(input_image_features, df)
    df_images = df.sort_values(by='Similarity_Images', ascending=False)
    top_three_images = df_images.head(3)
    
    # Calculate review similarity
    df['Similarity_Reviews'] = calculate_review_similarity(review_text, df)
    df_reviews = df.sort_values(by='Similarity_Reviews', ascending=False)
    top_three_reviews = df_reviews.head(3)
    
    # Calculate composite similarity score
    df['Composite_Similarity'] = (df['Similarity_Images'] + df['Similarity_Reviews']) / 2
    
    # Rank the pairs based on the composite similarity score
    df_combined = df.sort_values(by='Composite_Similarity', ascending=False)
    
    # Get top three most similar pairs
    top_three_combined = df_combined.head(3)
    
    # Output combined retrieval results
    for index, row in top_three_combined.iterrows():
        print("Pair", index+1)
        print("Image URL:", row['Image'])
        print("Review:", row['Review Text'])
        print("Cosine similarity of images:", row['Similarity_Images'])
        print("Cosine similarity of text:", row['Similarity_Reviews'])
        print("Composite similarity score:", row['Composite_Similarity'])
        print()  # Extra blank line for readability
else:
    print("Error extracting features from the input image.")  # Handle error if feature extraction fails
